In [150]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [151]:
df = pd.read_csv('summary_train_portfolios.csv')
df['ID'] = df['ID'].astype(int)
df.head()

,ID,VATD_NA,VATD_Passive,VATD_Alarm,VATD_Active,DMA_mean,DMA_std,VAY_mean,VAY_std,AP_mean,AP_std,AP_median,Vehicle_Passive_Restraint_count,Vehicle_Physical_Damage_Limit_mean,Vehicle_Physical_Damage_Limit_std,Vehicle_Collision_Coverage_Indicator_count,Less_Age,Vehicle_Driver_Points_mean,Vehicle_Driver_Points_std,ln_LR
0,1,0.571000,0.328000,0.079000,0.022000,46.321000,16.209216,7.070000,2.614179,332.251550,199.827994,323.300,731,49837.184874,34449.042079,601,88,0.121,0.398143,-1.518409
1,2,0.564000,0.332000,0.071000,0.033000,46.042000,15.900851,7.050000,2.562043,338.220410,212.877532,331.250,737,50630.914826,42628.495631,622,90,0.121,0.398143,1.228577
2,3,0.574000,0.320000,0.084000,0.022000,46.405000,16.050688,6.948000,2.644672,337.693300,218.700885,319.245,716,53091.482650,50575.378495,614,81,0.110,0.363362,0.038124
3,4,0.586333,0.311667,0.071667,0.030333,46.816667,15.997782,6.993333,2.629118,330.893983,208.687152,320.650,2183,49251.570133,34926.300808,1817,228,0.115,0.369442,-0.578894
4,5,0.586333,0.313333,0.073667,0.026667,46.331667,15.821518,7.080000,2.551425,336.671723,203.043195,329.405,2217,50658.578856,35814.482068,1846,236,0.127,0.401154,-0.349276


In [152]:
data = df.to_numpy()
features = data[:, :-1] 
target = data[:,-1]

In [61]:
# means = np.mean(features, axis=0)
# stds = np.std(features, axis = 0)
# print('Means of input features', means)
# print('Stds of input features', stds)

Means of input features [9.25000000e+01 5.84971160e-01 3.15576962e-01 7.43804105e-02
 2.50566129e-02 4.62023373e+01 1.59346976e+01 7.03348845e+00
 2.57643550e+00 3.35517357e+02 2.06342509e+02 3.27947554e+02
 1.69573913e+03 5.00561725e+04 3.56676588e+04 1.40611413e+03]
Stds of input features [5.31154403e+01 1.54170148e-02 1.42331847e-02 7.12295517e-03
 4.07845192e-03 3.77049890e-01 2.46995123e-01 8.55931687e-02
 4.18311996e-02 9.19216292e+00 1.03596012e+01 9.80451507e+00
 1.09089879e+03 9.96799350e+02 4.10472534e+03 9.05796007e+02]


In [62]:
# scaler = preprocessing.StandardScaler()
# scaler.fit(features)
# features_scaled = scaler.transform(features)
# print("Scaled - Means of input features: ", features_scaled.mean(axis=0))
# print("Scaled - Stds of input features: ", features_scaled.std(axis=0))

Scaled - Means of input features:  [ 0.00000000e+00 -5.24218350e-15  1.65930072e-15  6.17863248e-16
 -2.07563435e-16 -3.70235243e-15 -2.26871662e-15  5.64282920e-15
 -1.84876269e-15 -2.84796341e-15  1.11505008e-15 -6.66133815e-16
  1.11022302e-16  7.94050815e-16  5.59938569e-16  9.41276043e-17]
Scaled - Stds of input features:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [153]:
X_train = features[:147]
X_test = features[147:]
y_train = target[:147]
y_test = target[147:]

In [154]:
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)

print("Intercept:", linear_regression_model.intercept_)
print("Coefficients:", linear_regression_model.coef_)

Intercept: -211.6857019298588
Coefficients: [-2.18553049e-04  1.86337520e+02  1.92444178e+02  1.73077222e+02
  2.02907271e+02  1.75737432e-02  2.23388047e-01  2.44273328e-01
  3.29708705e-01 -1.17852451e-02  8.64941179e-03  3.16071037e-02
 -1.83605374e-03  1.39542704e-04  1.67564198e-06  2.77671368e-03
 -4.04722074e-03 -1.32368338e+01  7.20355109e+00]


In [155]:
linear_regression_pred = linear_regression_model.predict(X_train)
linear_regression_rmse = math.sqrt(mean_squared_error(y_train, linear_regression_pred))
print("Linear Regression - training RMSE:", linear_regression_rmse)

# Linear Regression - testing RMSE
lr_test_pred = linear_regression_model.predict(X_test)
lr_test_rmse = math.sqrt(mean_squared_error(y_test, lr_test_pred))
print("Linear Regression - testing RMSE:", lr_test_rmse)
MAE  = mean_absolute_error(y_test, lr_test_pred)
R2   = r2_score(y_test, lr_test_pred)
print("Root Mean Squared Error", lr_test_rmse)
print("Mean Absolute Error", MAE)
print("R-Squared Score", R2)

Linear Regression - training RMSE: 0.7378986584576234
Linear Regression - testing RMSE: 0.7611605864616392
Root Mean Squared Error 0.7611605864616392
Mean Absolute Error 0.6322351906728985
R-Squared Score 0.29273146378900583


In [156]:
from sklearn.linear_model import Lasso
dt_reg_model = Lasso(alpha=0.1)
dt_reg_model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.393e+01, tolerance: 9.629e-03
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [157]:
dt_reg_pred = dt_reg_model.predict(X_train)
dt_reg_rmse =math.sqrt (mean_squared_error(y_train, dt_reg_pred))
print("Decesion Tree Regression - training RMSE:", dt_reg_rmse)

# Linear Regression - testing RMSE
dt_reg_test_pred = dt_reg_model.predict(X_test)
dt_test_rmse = math.sqrt(mean_squared_error(y_test, dt_reg_test_pred))
print("Decesion Tree Regression - testing RMSE:", dt_test_rmse)

Decesion Tree Regression - training RMSE: 0.7518548092198962
Decesion Tree Regression - testing RMSE: 0.7734064178581826


In [158]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor

In [159]:
linear_params = {'fit_intercept': [True, False]}
ridge_params = {'alpha': [0.1, 1.0, 10.0]}
lasso_params = {'alpha': [0.1, 1.0, 10.0]}
knn_params = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
dt_params = {'max_depth': [5, 10, None]}
rf_params = {'n_estimators': [10, 50, 100]}
svm_params = {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'poly', 'rbf']}
xgb_params = {'learning_rate': [0.01, 0.1, 0.5], 
              'max_depth': [3, 5, 7],
              'subsample': [0.5, 0.8, 1.0],
              'colsample_bytree': [0.5, 0.8, 1.0],
              'gamma': [0, 0.1, 0.5],
              'n_estimators': [50, 100, 200]}

In [160]:
models = [
    (LinearRegression(), linear_params),
    (Ridge(), ridge_params),
    (Lasso(), lasso_params),
    (KNeighborsRegressor(), knn_params),
    (DecisionTreeRegressor(), dt_params),
    (RandomForestRegressor(), rf_params),
    (SVR(), svm_params)
#     (xgb.XGBRegressor(), xgb_params)
]

In [161]:
best_model = None
best_score = float('+inf')
for model, param_grid in models:
    grid_search = GridSearchCV(model, param_grid, scoring='neg_root_mean_squared_error')
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    score = math.sqrt(mean_squared_error(y_test, y_pred))
    print('{}: {:.2f}'.format(model.__class__.__name__, score))
    if score < best_score:
        best_score = score
        best_model = grid_search.best_estimator_

# print the best performing model and its parameters
print('Best model:', best_model)

LinearRegression: 0.76
Ridge: 0.77
Lasso: 0.77
KNeighborsRegressor: 0.93


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.680e+01, tolerance: 7.087e-03
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.724e+01, tolerance: 8.029e-03
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.907e+01, toleranc

DecisionTreeRegressor: 1.47
RandomForestRegressor: 0.94


KeyboardInterrupt: 

In [162]:
df_test = pd.read_csv('summary_test_portfolios.csv')

In [163]:
df_test.head()

,ID,VATD_NA,VATD_Passive,VATD_Alarm,VATD_Active,DMA_mean,DMA_std,VAY_mean,VAY_std,AP_mean,AP_std,AP_median,Vehicle_Passive_Restraint_count,Vehicle_Physical_Damage_Limit_mean,Vehicle_Physical_Damage_Limit_std,Vehicle_Collision_Coverage_Indicator_count,Less_Age,Vehicle_Driver_Points_mean,Vehicle_Driver_Points_std
0,1,0.587000,0.303000,0.088000,0.022000,46.328000,15.592997,7.076000,2.564918,331.804800,204.172601,319.06,733,49631.360332,33042.183930,604,84,0.110000,0.357810
1,2,0.584585,0.318318,0.065065,0.032032,45.958959,15.735645,7.095095,2.589087,325.044585,202.686500,307.40,717,51238.293444,36764.516727,584,83,0.095095,0.340879
2,3,0.599000,0.292000,0.086000,0.023000,46.017000,16.136328,7.086000,2.570839,343.187790,239.118194,322.24,729,49141.519251,28796.631737,605,87,0.121000,0.398143
3,4,0.597793,0.291876,0.085256,0.025075,45.947844,15.805586,7.219659,2.448033,335.759910,212.007607,328.60,737,51791.666667,39348.590568,619,90,0.127382,0.391709
4,5,0.572144,0.332665,0.068136,0.027054,45.998998,15.951751,7.016032,2.557218,343.425481,210.267342,334.96,742,50020.964361,34551.134909,617,90,0.132265,0.398771


In [164]:
lr_test_pred = dt_reg_model.predict(df_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


In [165]:
 df_test.head(25)

,ID,VATD_NA,VATD_Passive,VATD_Alarm,VATD_Active,DMA_mean,DMA_std,VAY_mean,VAY_std,AP_mean,AP_std,AP_median,Vehicle_Passive_Restraint_count,Vehicle_Physical_Damage_Limit_mean,Vehicle_Physical_Damage_Limit_std,Vehicle_Collision_Coverage_Indicator_count,Less_Age,Vehicle_Driver_Points_mean,Vehicle_Driver_Points_std
0,1,0.587000,0.303000,0.088000,0.022000,46.328000,15.592997,7.076000,2.564918,331.804800,204.172601,319.06,733,49631.360332,33042.183930,604,84,0.110000,0.357810
1,2,0.584585,0.318318,0.065065,0.032032,45.958959,15.735645,7.095095,2.589087,325.044585,202.686500,307.40,717,51238.293444,36764.516727,584,83,0.095095,0.340879
2,3,0.599000,0.292000,0.086000,0.023000,46.017000,16.136328,7.086000,2.570839,343.187790,239.118194,322.24,729,49141.519251,28796.631737,605,87,0.121000,0.398143
3,4,0.597793,0.291876,0.085256,0.025075,45.947844,15.805586,7.219659,2.448033,335.759910,212.007607,328.60,737,51791.666667,39348.590568,619,90,0.127382,0.391709
4,5,0.572144,0.332665,0.068136,0.027054,45.998998,15.951751,7.016032,2.557218,343.425481,210.267342,334.96,742,50020.964361,34551.134909,617,90,0.132265,0.398771
5,6,0.584000,0.306000,0.074000,0.036000,46.465000,15.834567,7.026000,2.534906,326.778890,186.989701,331.78,722,50572.916667,40716.615019,618,85,0.092000,0.337120
6,7,0.575000,0.328000,0.074000,0.023000,45.138000,15.073359,7.045000,2.567015,334.422920,197.231519,336.02,747,50152.310924,34032.501025,610,83,0.119000,0.380768
7,8,0.589178,0.309619,0.078156,0.023046,45.489980,16.125305,7.094188,2.509989,336.909389,213.569905,320.86,735,51605.263158,38796.811098,594,99,0.108216,0.388285
8,9,0.596192,0.301603,0.081162,0.021042,45.647295,15.871723,7.147295,2.490775,334.181373,200.493465,329.66,736,50973.084886,40203.660320,607,95,0.125251,0.404766
9,10,0.570140,0.325651,0.073146,0.031062,46.904810,16.249063,6.908818,2.698620,344.051483,201.055467,345.56,758,49480.789200,35627.212766,624,79,0.115230,0.352308


In [166]:
data_dict = {'Id': df_test.iloc[:, 0], 'ln_LR': lr_test_pred}

In [167]:
output_df = pd.DataFrame(data_dict)

In [168]:
output_df['Id'] = output_df['Id'].astype(int)
prefix = 'portfolio_'
output_df['Id'] = prefix + output_df['Id'].astype(str)

In [169]:
output_df.head()

,Id,ln_LR
0,portfolio_1,-0.949835
1,portfolio_2,-1.064548
2,portfolio_3,-0.689188
3,portfolio_4,-0.307577
4,portfolio_5,-0.409866


In [170]:
output_df.to_csv('./output5.csv', index=False)